In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install torch torchvision tqdm


In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm


In [4]:
BASE_DIR = "/content/drive/MyDrive/PlantDocBot"
train_dir = os.path.join(BASE_DIR, "data/train")
val_dir   = os.path.join(BASE_DIR, "data/val")


In [6]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


In [7]:
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
val_data   = datasets.ImageFolder(val_dir, transform=val_transforms)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=32, shuffle=False)

num_classes = len(train_data.classes)
print("Classes:", train_data.classes)
print("Total classes:", num_classes)


Classes: ['tomato_late_blight']
Total classes: 1


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = models.resnet18(weights="IMAGENET1K_V1")
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)


Using device: cpu
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 141MB/s]


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [10]:
EPOCHS = 5

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{EPOCHS}] | Loss: {train_loss:.4f} | Accuracy: {train_acc:.2f}%")


100%|██████████| 4/4 [00:40<00:00, 10.24s/it]


Epoch [1/5] | Loss: 0.0000 | Accuracy: 100.00%


100%|██████████| 4/4 [00:39<00:00,  9.96s/it]


Epoch [2/5] | Loss: 0.0000 | Accuracy: 100.00%


100%|██████████| 4/4 [00:37<00:00,  9.31s/it]


Epoch [3/5] | Loss: 0.0000 | Accuracy: 100.00%


100%|██████████| 4/4 [00:41<00:00, 10.41s/it]


Epoch [4/5] | Loss: 0.0000 | Accuracy: 100.00%


100%|██████████| 4/4 [00:36<00:00,  9.09s/it]

Epoch [5/5] | Loss: 0.0000 | Accuracy: 100.00%


In [11]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in tqdm(val_loader):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_acc = 100 * correct / total
print(f"Validation Accuracy: {val_acc:.2f}%")


100%|██████████| 2/2 [00:14<00:00,  7.03s/it]

Validation Accuracy: 3.03%


In [12]:
model_path = os.path.join(BASE_DIR, "models/plant_disease_resnet18.pth")
torch.save(model.state_dict(), model_path)
print("Model saved at:", model_path)


Model saved at: /content/drive/MyDrive/PlantDocBot/models/plant_disease_resnet18.pth
